# Imports e Funções

In [ ]:
%run '../codes/files/downloads.py'
%run '../codes/files/predict_encoding.py'
%run '../codes/files/create_folders.py'

import os
import pandas as pd

create_folders('')

In [ ]:
def write_results(df, filename):
    # Escreve em arquivo CSV    
    df.to_csv(filename,
              encoding='utf-8-sig',
              index=False,
              header=True,
              sep=';',
              decimal=',')
    
    return print('Tabela Salva!')

In [ ]:
def findfiles_ByExtensionContain(path, extension='.csv', contain='Indicadores'):
    """
    Function to find files with a especific extension in a path
    :param prefix:
    :param path:
    :param extension:
    :return:
    """
    
    from os import listdir
    filenames = listdir(path)
    lst = []
    
    for filename in filenames:
        if filename.endswith(extension) and contain in filename:
            lst.append(filename)
        
    return lst

# *Download* dos dados

In [ ]:
url = ['http://www.snis.gov.br/downloads/diagnosticos/ae/2018/Planilhas_AE2018.zip']

download_urls(url, 'data')

# *Unzip* arquivos

In [ ]:
%run '../codes/files/zip_file.py'

unzip_zipfiles('data')
unzip_zipfiles('data')

# Lê e Ajusta a Tabela de Indicadores

In [ ]:
list_type = findfiles_ByExtensionContain('data', extension='.xls', contain='Indicadores')

dfs = []

for i in list_type:
    file = os.path.join('data', i)
    
    df = pd.read_excel(file,
                       header=[0,1,2,3],
                       skiprows=range(0, 6))
    
    # Ajusta o Primeiro Nível do Cabeçalho
    a = df.columns.get_level_values(0)
    a = a.to_series()
    a = a.mask(lambda x: x.str.startswith('Unnamed')).ffill()
    a = a.replace('Tipo de serviço','PRESTADOR DE SERVIÇOS')
    a = a.str.title()
    a = a.replace({'  '     :' ',
                   ' E '    :' e ',
                   ' De '   :' de ',
                   ' Sobre ':' sobre '}, regex=True)
    a = a.to_list()

    # Ajusta o Segundo Nível do Cabeçalho
    b = df.columns.get_level_values(1)
    b = b.to_series()
    b = b.mask(lambda x: x.str.startswith('Unnamed'))
    b = b.fillna('Tipo de serviço')
    b = b.to_list()

    # Ajusta o Terceiro Nível do Cabeçalho
    c = df.columns.get_level_values(2)
    c = c.to_series()
    c = c.replace('-','')
    c = '('+c+')'
    c = c.replace('()','')
    c = c.to_list()

    # Ajusta o Quarto Nível do Cabeçalho
    d = df.columns.get_level_values(3)
    d = d.to_series()
    d = d.replace('-','')
    d = '('+d+')'
    d = d.replace('()','')
    d = d.to_list()

    # Junta tudo
    z = [x[0]+' '+x[1]+' '+x[2] for x in zip(b, c, d)]
    z = pd.Series(z)
    z = z.str.strip()
    z.to_list()

    # Define as Colunas
    #df.columns = [a, z]
    df.columns = z
    
    dfs.append(df)

dfs = pd.concat(dfs)
df = dfs

In [ ]:
df = df[df['UF'] == 'SP']

## Escreve a Tabela

In [ ]:
file = os.path.join('data', 'tab_Indicadores.csv')
write_results(df, file)

# Lê e Ajusta a Tabela de Informações

In [ ]:
list_type = findfiles_ByExtensionContain('data', extension='.xls', contain='Informacoes')

dfs = []

for i in list_type:
    file = os.path.join('data', i)
    
    df = pd.read_excel(file,
                       header=[0,1,2,3,4,5],
                       skiprows=range(0, 6))
    
    # Ajusta o Primeiro Nível do Cabeçalho
    a = df.columns.get_level_values(0)
    a = a.to_series()
    a = a.mask(lambda x: x.str.startswith('Unnamed')).ffill()    
    a = a.str.title()
    a = a.replace({'  '     :' ',
                   ' E '    :' e ',
                   ' De '   :' de ',
                   ' Com '   :' com ',
                   ' Sobre ':' sobre '}, regex=True)
    a = a.str.strip()
    a = a.to_list()
    
    # Ajusta o Segundo Nível do Cabeçalho
    b = df.columns.get_level_values(1)
    b = b.to_series()
    b = b.mask(lambda x: x.str.startswith('Unnamed')).ffill()
    b = b.str.title()
    b = b.replace({' À '   :' à ',
                   ' Ao '  :' ao ',
                   ' Ao '  :' ao ',
                   ' Da '  :' da ',
                   ' Das ' :' das ',
                   ' De '  :' de ',
                   ' Do '  :' do ',
                   ' Dos ' :' dos ',
                   ' Com ' :' com ',
                   ' A '   :' a ',
                   ' As '   :' as ',
                   ' E '   :' e ',
                   ' O '   :' o ',
                   ' Os '   :' os ',
                   ' Na '  :' na ',
                   ' No '  :' no ',
                   ' Nas ' :' nas ',
                   ' Não ' :' não ',
                   ' Em '  :' em ',
                   ' Pelo ':' pelo ',
                   ' Para ':' para ',
                   ' Qual ':' qual ',
                   ' Por ' :' por ',
                   ' Se '  :' se ',
                   ' Ou '  :' ou ',
                   '–':'-'}, regex=True)
    b = b.str.strip()
    b = b.replace('Uf','UF')
    b = b.fillna('')
    b = b.to_list()

    # Ajusta o Terceiro Nível do Cabeçalho
    c = df.columns.get_level_values(2)
    c = c.to_series()
    c = c.mask(lambda x: x.str.startswith('Unnamed'))
    c = c.fillna('')
    c = c.str.title()
    c = c.replace({' À '   :' à ',
                   ' Ao '  :' ao ',
                   ' Ao '  :' ao ',
                   ' Da '  :' da ',
                   ' Das ' :' das ',
                   ' De '  :' de ',
                   ' Do '  :' do ',
                   ' Dos ' :' dos ',
                   ' Com ' :' com ',
                   ' A '   :' a ',
                   ' E '   :' e ',
                   ' O '   :' o ',
                   ' Os '   :' os ',
                   ' Na '  :' na ',
                   ' No '  :' no ',
                   ' Nas ' :' nas ',
                   ' Não ' :' não ',
                   ' Em '  :' em ',
                   ' Pelo ':' pelo ',
                   ' Para ':' para ',
                   ' Qual ':' qual ',
                   ' Por ' :' por ',
                   ' Se '  :' se ',
                   ' Ou '  :' ou '}, regex=True)
    c = c.replace('-','')
    c = c.replace('()','')
    c = c.str.strip()
    c = c.to_list()

    # Ajusta o Quarto Nível do Cabeçalho
    d = df.columns.get_level_values(3)
    d = d.to_series()
    d = d.mask(lambda x: x.str.startswith('Unnamed'))
    d = d.fillna('')
    d = d.str.title()
    d = d.replace({' À '   :' à ',
                   ' Ao '  :' ao ',
                   ' Ao '  :' ao ',
                   ' Da '  :' da ',
                   ' Das ' :' das ',
                   ' De '  :' de ',
                   ' Do '  :' do ',
                   ' Dos ' :' dos ',
                   ' Com ' :' com ',
                   ' A '   :' a ',
                   ' As '   :' as ',
                   ' E '   :' e ',
                   ' O '   :' o ',
                   ' Na '  :' na ',
                   ' No '  :' no ',
                   ' Nas ' :' nas ',
                   ' Não ' :' não ',
                   ' Em '  :' em ',
                   ' Pelo ':' pelo ',
                   ' Para ':' para ',
                   ' Qual ':' qual ',
                   ' Por ' :' por ',
                   ' Se '  :' se ',
                   ' Ou '  :' ou '}, regex=True)    
    d = d.replace('()','')
    d = d.to_list()

    # Ajusta o Quinto Nível do Cabeçalho
    e = df.columns.get_level_values(4)
    e = e.to_series()
    e = e.mask(lambda x: x.str.startswith('Unnamed'))
    e = e.fillna('')
    e = e.replace('-','')
    e = '('+e+')'
    e = e.replace('()','')
    e = e.replace(' / ','/', regex=True)
    e = e.to_list()
    
    # Ajusta o Sexto Nível do Cabeçalho
    f = df.columns.get_level_values(5)
    f = f.to_series()
    f = f.mask(lambda x: x.str.startswith('Unnamed'))
    f = f.fillna('')
    f = f.replace('-','')
    f = '('+f+')'
    f = f.replace('()','')
    f = f.to_list()
    
    # Junta tudo
    z = [x[0]+' '+x[1]+' '+x[2]+' '+x[3]+' '+x[4] for x in zip(b, c, d, e, f)]
    z = pd.Series(z)
    z = z.replace({'/  /': '/',
                   '  '  :' '}, regex=True)
    z = z.str.strip()
    z.to_list()
    
    # Define as Colunas
    #df.columns = [a, z]
    df.columns = z
    
    dfs.append(df)

dfs = pd.concat(dfs)
df = dfs

In [ ]:
df = df[df['UF'] == 'SP']

## Escreve a Tabela

In [ ]:
file = os.path.join('data', 'tab_Informações.csv')
write_results(df, file)

# Git

In [ ]:
%run '~/Documents/Geodata/SourceCode/codes/git/update_github.py'

git_full('/home/michel/Documents/Geodata/SourceCode/data_SNIS', '.', 'Atualizando')